# <center>Assignment for News Virality Prediction</center>

## __Problem Statement__: 
- Aim of this assignment is to predict the virality of __news articles__.


<br>

__How to approach this problem__

- To achieve first we gonna scrap the news data from the indiatimes website using beautiful soup.

- To train a machine learning model will use the UCI news data. Apply that trained model to predict the virality.

- Here we explained the term virality with the number of shares(how many times does the article is shared) article gonna have.




To scrap and train the model i am using Google collab so i directly all the requied library in my notebook but in case of jupyter notebook, need to intall through 'conda' or 'pip' from your cmd.

In [3]:
# Install the newspaper library
! pip install newspaper3k

     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 7.4MB 36kB/s 
     |████████████████████████████████| 194kB 39.9MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=f94b4c53abc4089e5ccecb1aad077c3115af88cc8ee1d100ff3a5a5b6e82e2ad
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=526adf9626b79ebdfc1ba43c2d02bed3e562487a4956f1586b7c47bc0351d502
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=f298a334c9cb8e49369b1881a4ab0ba742f74293252583eb34291cc1b8dec6ae
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81

In [0]:
# import all the required libraries
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np

To work with the text data we need to import Natural language took kit library i.e nltk

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Start the Scraping

In [0]:
news_url = "https://timesofindia.indiatimes.com/world"
r = requests.get(news_url)

In [0]:
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.findAll('a', attrs = {'class':'w_img'})

In [0]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

Create a list to store all the data formates in the form of dictionary.

In [0]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [10]:
# check the list
df

[{'Keywords': ['cooperation',
   'promoting',
   'trump',
   'statement',
   'russia',
   'moscow',
   'joint',
   'intelligence',
   'rare',
   'officials',
   'russian',
   'putin',
   'issue',
   'issued'],
  'Summary': 'Reuters photoDownload The Times of India News App for Latest World NewsSubscribe Start Your Daily Mornings with Times of India Newspaper!\nOrder NowWASHINGTON: US President Donald Trump and his Russian counterpart Vladimir Putin issued a rare joint statement on Saturday commemorating a 1945 World War Two link-up of US and Soviet troops on their way to defeat Nazi Germany as an example of how their countries can cooperate.The statement by Trump and Putin comes amid deep strains in US-Russian ties over a raft of issues, from arms control and Russia’s intervention in Ukraine and Syria to US charges that Russia has spread disinformation about the novel coronavirus pandemic and interfered in US election campaigns.The Wall Street Journal reported that the decision to issu

In [11]:
# convert the dictionary data to a pandas dataframe

dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,Trump and Putin issue rare joint statement pro...,Reuters photo\n\nDownload The Times of India N...,Reuters photoDownload The Times of India News ...,"[cooperation, promoting, trump, statement, rus..."
1,Covid-19 pandemic is far from over: WHO,"Apr 28, 2020, 05:08PM IST\n\nSource: AP\n\nThe...","Apr 28, 2020, 05:08PM ISTSource: APThe head of...","[far, countries, cases, europe, covid19, tedro..."
2,Covid-19 in UK: We are beginning to turn the t...,"Apr 28, 2020, 05:06PM IST\n\nSource: AP\n\nPri...","Apr 28, 2020, 05:06PM ISTSource: APPrime Minis...","[week, country, work, life, tide, turn, beginn..."
3,US: First responders return to work after Covi...,"Apr 28, 2020, 05:04PM IST\n\nSource: AP\n\nAcr...","Apr 28, 2020, 05:04PM ISTSource: APAcross the ...","[ziman, work, contracted, york, worries, puts,..."
4,Covid-19: Testing won't 'be a problem' for reo...,"Apr 28, 2020, 08:40AM IST\n\nSource: AP\n\nThe...","Apr 28, 2020, 08:40AM ISTSource: APThe White H...","[president, getting, trump, problem, white, un..."


## <center><b>Machine Leanring Model for predicting virality of news</b></center>

In [0]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split

In [0]:
# To train the model we used the UCI news data which is available at the following link.
FILEPATH="https://raw.githubusercontent.com/AIVenture0/News-Virality-Prediction-using-Machine-Learning/master/OnlineNewsPopularity.csv"

- Transform the data to train the model

In [0]:
def clean_cols(data):
    """Clean the column names by stripping and lowercase."""
    clean_col_map = {x: x.lower().strip() for x in list(data)}
    return data.rename(index=str, columns=clean_col_map)

def TrainTestSplit(X, Y, R=0, test_size=0.2):
    """Easy Train Test Split call."""
    return train_test_split(X, Y, test_size=test_size, random_state=R)

- I removed these features below because some of them are not available in our UCI training data. which we are using for training the model.

In [0]:
full_data = clean_cols(pd.read_csv(FILEPATH))
train_set, test_set = train_test_split(full_data, test_size=0.20, random_state=42)

x_train = train_set.drop(['url','shares', 'timedelta', 'lda_00','lda_01','lda_02','lda_03','lda_04','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'timedelta', 'num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_test = test_set['shares']

In [16]:
clf=XGBRegressor(random_state=32,max_depth=5,n_estimators=1000)
clf.fit(x_train, y_train)

[17:57:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=32,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
rf_res = pd.DataFrame(clf.predict(x_train),list(y_train))

In [18]:
rf_res.reset_index(level=0, inplace=True)
rf_res_df = rf_res.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
rf_res_df.head()

,Actual shares,Predicted shares
0,16100,4824.281250
1,508,813.780640
2,1300,3281.530762
3,3100,4311.856445
4,6900,5627.784668


### Converting Crawled News according to Training Set in UCI Dataset

In [19]:
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:

def rate_unique(words):
    words=tokenize(words)
    no_order = list(set(words))
    rate_unique=len(no_order)/len(words)
    return rate_unique
    
def rate_nonstop(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    rate_nonstop=len(filtered_sentence)/len(words)
    no_order = list(set(filtered_sentence))
    rate_unique_nonstop=len(no_order)/len(words)
    return rate_nonstop,rate_unique_nonstop

def avg_token(words):
    words=tokenize(words)
    length=[]
    for i in words:
        length.append(len(i))
    return np.average(length)

In [0]:
from textblob import TextBlob

In [25]:
# intall the datefinder.
!pip install datefinder

In [0]:
import datefinder
import datetime  
from datetime import date 
def day(article_text):
    article=article_text
    if len(list(datefinder.find_dates(article)))>0:
        date=str(list(datefinder.find_dates(article))[0])
        date=date.split()
        date=date[0]
        year, month, day = date.split('-')     
        day_name = datetime.date(int(year), int(month), int(day)) 
        return day_name.strftime("%A")
    return "Monday"

In [0]:
def tokenize(text):
    text=text
    return word_tokenize(text)

In [0]:
pos_words=[]
neg_words=[]
def polar(words):
    all_tokens=tokenize(words)
    for i in all_tokens:
        analysis=TextBlob(i)
        polarity=analysis.sentiment.polarity
        if polarity>0:
            pos_words.append(i)
        if polarity<0:
            neg_words.append(i)
    return pos_words,neg_words

In [0]:
def rates(words):
    words=polar(words)
    pos=words[0]
    neg=words[1]
    all_words=words
    global_rate_positive_words=(len(pos)/len(all_words))/100
    global_rate_negative_words=(len(neg)/len(all_words))/100
    pol_pos=[]
    pol_neg=[]
    for i in pos:
        analysis=TextBlob(i)
        pol_pos.append(analysis.sentiment.polarity)
        avg_positive_polarity=analysis.sentiment.polarity
    for j in neg:
        analysis2=TextBlob(j)
        pol_neg.append(analysis2.sentiment.polarity)
        avg_negative_polarity=analysis2.sentiment.polarity
    min_positive_polarity=min(pol_pos)
    max_positive_polarity=max(pol_pos)
    min_negative_polarity=min(pol_neg)
    max_negative_polarity=max(pol_neg)
    avg_positive_polarity=np.average(pol_pos)
    avg_negative_polarity=np.average(pol_neg)
    return global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity

In [0]:
df2=[]
for i in news:
    pred_info={}
    article = Article(i, language="en") # en for English 
    article.download() 
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(tokenize(article.title))
    pred_info['n_tokens_content']=len(tokenize(article.text))
    pred_info['n_unique_tokens']=rate_unique(article.text)
    pred_info['n_non_stop_words']=rate_nonstop(article.text)[0]
    pred_info['n_non_stop_unique_tokens']=rate_nonstop(article.text)[1]
    pred_info['num_hrefs']=article.html.count("https://timesofindia.indiatimes.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=avg_token(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    if "life-style" in article.url:
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    
    if day(i)=="Monday":
        pred_info['weekday_is_monday']=1
    else:
        pred_info['weekday_is_monday']=0
    if day(i)=="Tuesday":
        pred_info['weekday_is_tuesday']=1
    else:
        pred_info['weekday_is_tuesday']=0
    if day(i)=="Wednesday":
        pred_info['weekday_is_wednesday']=1
    else:
        pred_info['weekday_is_wednesday']=0
    if day(i)=="Thursday":
        pred_info['weekday_is_thursday']=1
    else:
        pred_info['weekday_is_thursday']=0
    if day(i)=="Friday":
        pred_info['weekday_is_friday']=1
    else:
        pred_info['weekday_is_friday']=0
    if day(i)=="Saturday":
        pred_info['weekday_is_saturday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_saturday']=0
    if day(i)=="Sunday":
        pred_info['weekday_is_sunday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_sunday']=0
        pred_info['is_weekend']=0
        
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=rates(article.text)[0]
    pred_info['global_rate_negative_words']=rates(article.text)[1]
    pred_info['avg_positive_polarity']=rates(article.text)[2]
    pred_info['min_positive_polarity']=rates(article.text)[3]
    pred_info['max_positive_polarity']=rates(article.text)[4]
    pred_info['avg_negative_polarity']=rates(article.text)[5]
    pred_info['min_negative_polarity']=rates(article.text)[6]
    pred_info['max_negative_polarity']=rates(article.text)[7]    
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    df2.append(pred_info)

In [31]:
pred_df=pd.DataFrame(df2)
pred_test=pred_df.drop(['text'],axis=1)
pred_df.head()

,text,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity
0,Reuters photo\n\nDownload The Times of India N...,9,400,0.555000,0.667500,0.467500,231,13,0,5.052500,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0.404762,0.078571,0.025,0.01,0.340000,0.1,0.5,-0.600000,-0.600000,-0.6,0.900000,0.30
1,"Apr 28, 2020, 05:08PM IST\n\nSource: AP\n\nThe...",8,253,0.636364,0.691700,0.490119,185,9,0,4.351779,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.551094,0.132183,0.265,0.05,0.288320,0.1,0.5,-0.648980,-0.714286,-0.6,1.000000,0.10
2,"Apr 28, 2020, 05:06PM IST\n\nSource: AP\n\nPri...",15,229,0.663755,0.655022,0.515284,185,9,0,4.305677,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.226105,-0.063520,0.735,0.13,0.260606,0.1,0.5,-0.422826,-0.714286,-0.1,0.000000,0.00
3,"Apr 28, 2020, 05:04PM IST\n\nSource: AP\n\nAcr...",9,323,0.640867,0.650155,0.482972,185,9,0,4.269350,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.342922,0.081739,0.890,0.33,0.263076,0.1,0.5,-0.355371,-0.714286,-0.1,0.333333,0.25
4,"Apr 28, 2020, 08:40AM IST\n\nSource: AP\n\nThe...",15,152,0.736842,0.684211,0.539474,185,9,0,4.486842,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0.342472,0.174148,1.280,0.50,0.283119,0.1,0.7,-0.330306,-0.714286,-0.1,0.000000,0.00


Final Results depicting the Likelihood of Virality of News

In [32]:
test2=pd.DataFrame(clf.predict(pred_test),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,Reuters photo\n\nDownload The Times of India N...,2175.422119
1,"Apr 28, 2020, 05:08PM IST\n\nSource: AP\n\nThe...",11031.876953
2,"Apr 28, 2020, 05:06PM IST\n\nSource: AP\n\nPri...",18665.517578
3,"Apr 28, 2020, 05:04PM IST\n\nSource: AP\n\nAcr...",26694.609375
4,"Apr 28, 2020, 08:40AM IST\n\nSource: AP\n\nThe...",42809.589844
5,"Apr 28, 2020, 08:43AM IST\n\nSource: AP\n\nWit...",59673.289062
6,"Apr 28, 2020, 08:39AM IST\n\nSource: AP\n\nNor...",23765.824219
7,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",25147.902344
8,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",57684.828125
9,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",30553.755859


Here we got the virality of different news based on Xgb regressor model. This is not the best because our training data is of different news categories and also the timing of news. 

- Currently all the news related to COVID-19 has need to more shares. But in our training data that kind of data is not present. So it is not a very reliable model.